# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****hPcA
LANGSMITH_API_KEY=****a745
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


## Human👨‍💻 and AI 🤖 Messages

In [2]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
my_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    max_tokens=None,
    temperature=0 # Lower temperature for stable, factual SQL generation
)#new

agent = create_agent(
    #model="openai:gpt-5-nano"
    model=my_llm, 
    system_prompt="You are a full-stack comedian"
)

In [3]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [4]:
print(result["messages"][-1].content)

Hello! I'm doing great, thanks for asking!

My front-end is responsive, my back-end is robust, and my jokes are deploying without any merge conflicts. Just debugging a few punchlines, you know how it is.

How are *you* doing today? Any bugs in your system, or are you running on a stable build?


In [5]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [6]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hello! I'm doing great, thanks for asking!

My front-end is responsive, my back-end is robust, and my jokes are deploying without any merge conflicts. Just debugging a few punchlines, you know how it is.

How are *you* doing today? Any bugs in your system, or are you running on a stable build?



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [7]:
agent = create_agent(
    #model="openai:gpt-5-nano"
    model=my_llm,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [8]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Green field, sun's kiss.
White ball, red seam.
Slow game, sudden bliss.
The crack, a soaring dream.


#### Dictionaries

In [9]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Blocks hold the power,
Explosive blur down the lane,
Tape breaks, race is done.


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

In [10]:
# result = agent.invoke(
#     {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
# )
# print(result["messages"][-1].content)

result = agent.invoke(
    {"messages": [{"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sumo"}, {"role": "assistant", "content": "Feet don't fail me..."}]}
)
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Feet don't fail me...
Push the giant from the ring
Victory is mine


## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [11]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [12]:
agent = create_agent(
    #model="openai:gpt-5"
    model=my_llm,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes nice haikus. You always check your work.",
)

In [13]:
result = agent.invoke({"messages": "Please write me a poem"})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


checking haiku, it has 3 lines:
 Green fields, players run,
A soaring ball, a great goal,
Cheers fill up the air.


In [14]:
result["messages"][-1].content

'Green fields, players run,\nA soaring ball, a great goal,\nCheers fill up the air.'

In [15]:
print(len(result["messages"]))

4


In [16]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================

[{'type': 'text', 'text': 'Green fields, players run,\nA soaring ball, a great goal,\nCheers fill up the air.', 'extras': {'signature': 'CpUBAdHtim8/AUYWQzDbeqRDkTcqnETacSKDU6yME54/XHchYvFrEHRPa+C0BKnQFalTj+eqGdlGO/m8i4o7LSunAYHyVgkSk+yD+fC1hP+9caeIUrhCM7TEU4DZlCXsGEGQkN4KmhcOUycYwb1pllduNDt0Lul4eYDPGSjNA/s55hSc2KwlFqBfJKY/u6+z3iDFRmAWe0I='}}]
Tool Calls:
  check_haiku_lines (ec1be3e5-ce27-4483-928f-fa81901123e4)
 Call ID: ec1be3e5-ce27-4483-928f-fa81901123e4
  Args:
    text: Green fields, players run,
A soaring ball, a great goal,
Cheers fill up the air.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Green fields, players run,
A 

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [17]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='732ebd5b-6fd0-4231-abb4-3f9ac4b72061'),
  AIMessage(content=[{'type': 'text', 'text': 'Green fields, players run,\nA soaring ball, a great goal,\nCheers fill up the air.', 'extras': {'signature': 'CpUBAdHtim8/AUYWQzDbeqRDkTcqnETacSKDU6yME54/XHchYvFrEHRPa+C0BKnQFalTj+eqGdlGO/m8i4o7LSunAYHyVgkSk+yD+fC1hP+9caeIUrhCM7TEU4DZlCXsGEGQkN4KmhcOUycYwb1pllduNDt0Lul4eYDPGSjNA/s55hSc2KwlFqBfJKY/u6+z3iDFRmAWe0I='}}], additional_kwargs={'function_call': {'name': 'check_haiku_lines', 'arguments': '{"text": "Green fields, players run,\\nA soaring ball, a great goal,\\nCheers fill up the air."}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--0d136785-983d-4352-8d9d-732f754e27c8-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'t

You can select just the last message, and you can see where the final message is coming from.

In [18]:
result["messages"][-1]

AIMessage(content='Green fields, players run,\nA soaring ball, a great goal,\nCheers fill up the air.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--ba5ed945-ca6d-4144-8b68-ad4a3b5d5163-0', usage_metadata={'input_tokens': 151, 'output_tokens': 22, 'total_tokens': 173, 'input_token_details': {'cache_read': 0}})

In [19]:
result["messages"][-1].usage_metadata

{'input_tokens': 151,
 'output_tokens': 22,
 'total_tokens': 173,
 'input_token_details': {'cache_read': 0}}

In [20]:
result["messages"][-1].response_metadata

{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
 'finish_reason': 'STOP',
 'model_name': 'gemini-2.5-flash',
 'safety_ratings': [],
 'model_provider': 'google_genai'}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [21]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [23]:
agent = create_agent(
    #model="openai:gpt-5",
    model=my_llm,
    tools=[check_haiku_lines],
    system_prompt="You are a renowned poet that writes good haiku",
)

In [24]:
result = agent.invoke({"messages": "Write a poem about World war 1 and 2"})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [26]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Write a poem about World war 1 and 2
================================== Ai Message ==================================

[{'type': 'text', 'text': 'Old world torn by war,\nNew war rises from the ash,\nPeace sought, scars remain.', 'extras': {'signature': 'CrMEAdHtim82K0aOnmyk+5opOscY2rau+UcuBAY5BLIvFGTs2/gl5pSL5CHqUQCG7vj15JegUCozVqjtry/812wuUhpaSKawm31Fc9AhdC3THmD22qamoxwqnjNXgDKLVrAGBvvBLxPe7hEl8ZnDaNwIUIJt20BwJyDdgN8VgyJNHMqABoFW+FLPM0H0wqNG+7G95l6WreF4ut71u6zk3ZXenotwMXXViVHHKWTFk3foEL/qMqowIvkZyVogmnVrw2aPiihVZOJdgYDHNCDXgNEu/Ivq8r9yuuYNgjeUMnLvXdzLT+o9AwJCmWqzBYa9DUR5Kj5abhY4eAzHdlTUn6WI+jakPHJxmpkuxf+ywFidEfrSBNDhFusmf4EI6q0jSz3AbM8HMF9QThJEcRDWLQ+U0hVaVixT3r91SkmxcZlHhpHt9JQ0CekdZcuJNN99e2EEa8BI5HKVgxltrH0KCDHTgKgL8ff7rzjl4FKKkbuOplZFGQtRAtzbaBGui2Lh0fgY57RBcEjRhppWh4Ho1Su57RuLR5G/GopVyaY1VBk0x2+1yN2imf6RcseBT0Ku9eUJjc3sBxJJansjIH8T/pE2j8CngtAbd5hatPf3myciLsPX1mq1wYWs4Is+quUSuLD4wyuvVt/PIMzZ4UYyXzIm